In [2]:
print ("Mounting Google Drive to This Notebook ...")

from google.colab import drive
drive.mount('/content/drive')

print ("Successfully Mounted Google Drive to this notebook!")

Mounting Google Drive to This Notebook ...
Mounted at /content/drive
Successfully Mounted Google Drive to this notebook!


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import data_table

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Spotify/playlist_summary_external.txt',sep='\t')
%reload_ext google.colab.data_table
df.columns

Index(['playlist_uri', 'owner', 'streams', 'stream30s', 'dau', 'wau', 'mau',
       'mau_previous_month', 'mau_both_months', 'users', 'skippers',
       'owner_country', 'n_tracks', 'n_local_tracks', 'n_artists', 'n_albums',
       'monthly_stream30s', 'monthly_owner_stream30s', 'tokens', 'genre_1',
       'genre_2', 'genre_3', 'mood_1', 'mood_2', 'mood_3'],
      dtype='object')

In [5]:
# Remove Spotify Owned Playlists

df = df[df['owner']!='spotify']

In [6]:
# Create Dependent Variables

# Variable 1 - Percentage of Today's Total Streams Over 30 seconds in duration

df['quality_streams'] = (df['stream30s']/df['streams'])*100
df['quality_streams'].fillna(0,inplace=True)

# Variable 2 - Growth Percentage (Ratio of Past to Previous Month Users)

def generate_growth_variable(x,y):
  if y==0:
    return np.nan
  else:
    #return x/y
    return ((x-y)/y)*100

#Growth Rate Absolute

df['growth_absolute'] = df['mau']-df['mau_previous_month']
#df['growth_variable'].fillna(0,inplace=True)
df['growth_absolute'].dropna(inplace=True)

# Variable 3 - Quality Users ('WAU','DAU','MAU')


In [ ]:
# Quad Growth Absolute - All

fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)
#ax.set(xscale="log")
#ax.set(yscale="log")
#ax.set(ylim=(-100, 100))
#ax.grid(False)
ax.set(xlabel='% of Daily Total Streams > 30 Seconds',ylabel='MAU Monthly Growth Rate')

#sns.set(font_scale = 500,font="Arimo")
sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white',
            'ytick.labelsize' : '40', 'xtick.labelsize':'40' })

#sns.set(rc={'font.size': 100, 'axes.labelsize': 100, 'legend.fontsize': 100, 'axes.titlesize': 100, 'xtick.labelsize': 100, 'ytick.labelsize': 100})

#sns.scatterplot(data=new_df,x=combo[0],y=combo[1],size = combo[2],sizes=(5, 2000),x_jitter=10,y_jitter=10)
sns.scatterplot(data=df,x='quality_streams',y='growth_absolute',size='mau',sizes=(200, 10000),x_jitter=0.1,y_jitter=0.1,ax=ax,
                alpha= 0.5,color="#00e091")
plt.axhline(y=0, color='#EFE9F4', linestyle='--', linewidth=5)           
plt.axvline(x=df['quality_streams'].max()/2, color='#EFE9F4',linestyle='--', linewidth=5) 
#plt.legend([],[], frameon=False)
plt.legend(bbox_to_anchor=(1, 1,5,5), loc=2,fontsize='x-large', title_fontsize='40')
plt.xlabel('% of Daily Total Streams > 30 Seconds')
plt.ylabel('MAU Monthly Growth Rate')
#plt.rcParams["axes.labelsize"] = 1000
#plt.savefig('quadrant.pdf')

In [ ]:
# Quad Growth Absolute - With Cutoff

df2 = df[df['growth_absolute']>=-20000]
df2 = df2[df2['growth_absolute']<=20000]

fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)
#ax.set(xscale="log")
#ax.set(yscale="log")
#ax.set(ylim=(-100, 100))
#ax.grid(False)
ax.set(xlabel='% of Daily Total Streams > 30 Seconds',ylabel='MAU Monthly Growth Rate')

#sns.set(font_scale = 500,font="Arimo")
sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white',
            'ytick.labelsize' : '40', 'xtick.labelsize':'40' })

#sns.set(rc={'font.size': 100, 'axes.labelsize': 100, 'legend.fontsize': 100, 'axes.titlesize': 100, 'xtick.labelsize': 100, 'ytick.labelsize': 100})

#sns.scatterplot(data=new_df,x=combo[0],y=combo[1],size = combo[2],sizes=(5, 2000),x_jitter=10,y_jitter=10)
sns.scatterplot(data=df2,x='quality_streams',y='growth_absolute',size='mau',sizes=(200, 10000),x_jitter=0.1,y_jitter=0.1,ax=ax,
                alpha= 0.5,color="#00e091")
plt.axhline(y=0, color='#EFE9F4', linestyle='--', linewidth=5)           
plt.axvline(x=df2['quality_streams'].max()/2, color='#EFE9F4',linestyle='--', linewidth=5) 
plt.legend([],[], frameon=False)
plt.xlabel('% of Daily Total Streams > 30 Seconds')
plt.ylabel('MAU Monthly Growth Rate')
#plt.rcParams["axes.labelsize"] = 1000
#plt.savefig('quadrant.pdf')

In [ ]:
# Case Study 1
#df[df['playlist_uri']=='spotify:user:2ed777bdb3b70101b69430fedc051d90:playlist:1bNVBDbxwB5sPEW3GJaoSu'][['tokens','growth_absolute','quality_streams']]

# Case Study 2
#df[df['playlist_uri']=='spotify:user:a2e9efe4e9ec6085a112881a50650d51:playlist:5rMh5sksZqzbXdbnEYw1g6'][['tokens','growth_absolute','quality_streams']]

# Case Study 3
#df[df['playlist_uri']=='spotify:user:e60d19313a8df36c7e6cf59e259e0d66:playlist:19PgP2QSGPcm6Ve8VhbtpG'][['tokens','growth_absolute','quality_streams']]


In [30]:
# Function to Plot Normalized
# https://re-thought.com/exploring-correlation-in-python/

from scipy import stats
import scipy
import numpy as np

def plot_normalized(mydf,x_val,y_val,h,w,z):
  mydf_small= mydf[[x_val,y_val]]
  z_scores = stats.zscore(mydf_small)

  abs_z_scores = np.abs(z_scores)
  filtered_entries = (abs_z_scores < z).all(axis=1)
  new_df = mydf_small[filtered_entries]

  corr,pvalue = scipy.stats.pearsonr(new_df[x_val],new_df[y_val])
  corr2,pvalue2 = scipy.stats.spearmanr(new_df[x_val],new_df[y_val])
  corr3, pvalue3 = scipy.stats.kendalltau(new_df[x_val],new_df[y_val])

  fig_dims = (h, w)
  fig, ax = plt.subplots(figsize=fig_dims)
  #ax.set(xscale="log")
  #ax.set(yscale="log")

  #sns.set(font_scale = 500,font="Arimo")
  sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white',
              'ytick.labelsize' : '40', 'xtick.labelsize':'40' })

  sns.regplot(data=new_df,x=x_val,y=y_val,fit_reg=True,x_jitter=.5,y_jitter=.5,ax=ax,scatter_kws={"color": "#00e091"}, line_kws={"color": "#EFE9F4"})

  print ("Pearson Coefficient: {}".format(corr))
  print ("Pearson P-Value: {}".format(pvalue))
  print ("Pearson P-Value: {:0.300f}".format(pvalue))
  print ("\n")

In [235]:
def count_genre(x,y,z):
  cnt=0
  if x!='-':
    cnt+=1
  if y!='-':
    cnt+=1  
  if z!='-':
    cnt+=1
  return cnt

df['genre_count']=df.apply(lambda row: count_genre(row['genre_1'],row['genre_2'],row['genre_3']),axis=1)

In [ ]:
indep_variables = ['genre_count']
dep_variables = ['mau','quality_streams','growth_absolute']

for i in dep_variables:
  for j in indep_variables:
    plot_normalized(df,j,i,10,10,1)

In [7]:
# Playlist Titles Matter
# More Generic, More Ubiqitous, Search Optimization
# Uniqueness
# Lexical Complexity (pip install textstat)

tokens = pd.Series(df['tokens']).explode()
tokens = [x.strip('][').split(', ') for x in df['tokens']]
tokens2 = sum(tokens, [])

from collections import Counter

token_counter = Counter(tokens2)

token_counter_df = pd.DataFrame.from_records(list(dict(token_counter).items()), columns=['token','count'])

token_counter_df2 = token_counter_df.sort_values('count',ascending=False).reset_index()
token_counter_df2 = token_counter_df2[token_counter_df2['token']!='']

df['tokens_list']=tokens

In [254]:
bottom_10k = token_counter_df2.tail(10000)['token'].tolist()
first_10k = token_counter_df2.head(10000)['token'].tolist()

def get_top_count(x,y):
  cnt=0
  for i in y:
    if x.count(i)>0:
      cnt+=1
  return cnt

df['top_10k']=df.apply(lambda row: get_top_count(row['tokens'],first_10k),axis=1)
df['bottom_10k']=df.apply(lambda row: get_top_count(row['tokens'],bottom_10k),axis=1)

In [267]:
df.columns

Index(['playlist_uri', 'owner', 'streams', 'stream30s', 'dau', 'wau', 'mau',
       'mau_previous_month', 'mau_both_months', 'users', 'skippers',
       'owner_country', 'n_tracks', 'n_local_tracks', 'n_artists', 'n_albums',
       'monthly_stream30s', 'monthly_owner_stream30s', 'tokens', 'genre_1',
       'genre_2', 'genre_3', 'mood_1', 'mood_2', 'mood_3', 'quality_streams',
       'growth_absolute', 'Owner_Count', 'genre_in_token', 'top_100',
       'tokens_list', 'top_100_log', 'top_10k', 'bottom_10k'],
      dtype='object')

In [274]:
df['top_10k_perc']=(df['top_10k']/df['tokens_list'].apply(len))*100
df['bottom_10k_perc']=(df['bottom_10k']/df['tokens_list'].apply(len))*100


In [ ]:
fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)

import matplotlib.ticker as mtick

ax.xaxis.set_major_formatter(mtick.PercentFormatter())

sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white','ytick.labelsize' : '40', 'xtick.labelsize':'40' })
sns.scatterplot(data=df[df['mau']<50000],x='top_10k_perc',y='mau',ax=ax,color="#00e091")
plt.legend([],[], frameon=False)

In [ ]:
fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)

import matplotlib.ticker as mtick

ax.xaxis.set_major_formatter(mtick.PercentFormatter())

sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white','ytick.labelsize' : '40', 'xtick.labelsize':'40' })
sns.scatterplot(data=df[df['mau']<50000],x='bottom_10k_perc',y='mau',ax=ax,color="#00e091")
plt.legend([],[], frameon=False)

In [34]:
owner_playlist_count = df.groupby('owner',as_index=False).agg({'playlist_uri': ['count']}).reset_index()
owner_playlist_count = pd.DataFrame({'owner': owner_playlist_count['owner'],'Owner_Count':owner_playlist_count['playlist_uri','count']})

df=df.merge(owner_playlist_count,on='owner',suffixes=('','owner_'))

In [ ]:
# Number of Genres vs. MAU

fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)
#ax.set(xscale="log")
#ax.set(yscale="log")
#ax.set(xlim=(0,1))
#ax.grid(False)
ax.set(xlabel='% of Daily Total Streams > 30 Seconds',ylabel='MAU Monthly Growth Rate')

#sns.set(font_scale = 500,font="Arimo")
sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white',
            'ytick.labelsize' : '40', 'xtick.labelsize':'40' })

sns.scatterplot(data=df,y='Owner_Count',x='growth_absolute',ax=ax,color="#00e091")
plt.legend([],[], frameon=False)
plt.xlabel('% of Daily Total Streams > 30 Seconds')
plt.ylabel('MAU Monthly Growth Rate')
#plt.rcParams["axes.labelsize"] = 1000
#plt.savefig('quadrant.pdf')


In [5]:
genre_1, genre_2, genre_3 = df[['owner','genre_1']],df[['owner','genre_2']],df[['owner','genre_3']]
genre_1.columns,genre_2.columns,genre_3.columns=['owner','genre'],['owner','genre'],['owner','genre']

genre_groups = genre_1.append(genre_2).append(genre_3)
genre_groups = genre_groups[genre_groups['genre']!='-']
genre_groups.drop_duplicates(inplace=True)

genre_groups.head(5)

genre_agg = genre_groups.groupby('owner')['genre'].nunique().reset_index()

In [ ]:
fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)
#ax.set(xscale="log")
#ax.set(yscale="log")
ax.set(xlim=(0,18))
#ax.grid(False)
ax.set(xlabel='% of Daily Total Streams > 30 Seconds',ylabel='MAU Monthly Growth Rate')

#sns.set(font_scale = 500,font="Arimo")
sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white',
            'ytick.labelsize' : '40', 'xtick.labelsize':'40' })

sns.scatterplot(data=genre_plot[genre_plot['mau']<=50000],y='mau',x='genre',ax=ax,color="#00e091")

plt.legend([],[], frameon=False)
plt.xlabel('% of Daily Total Streams > 30 Seconds')
plt.ylabel('MAU Monthly Growth Rate')


In [ ]:
genre_line_plot = genre_plot.groupby('genre').agg({'quality_streams':'mean'}).reset_index()

fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)
#ax.set(xscale="log")
#ax.set(yscale="log")
ax.set(xlim=(0,18))
#ax.grid(False)
ax.set(xlabel='% of Daily Total Streams > 30 Seconds',ylabel='MAU Monthly Growth Rate')

#sns.set(font_scale = 500,font="Arimo")
sns.set(rc={'figure.facecolor': '#182722','axes.facecolor':'#182722','xtick.color':'white', 'ytick.color':'white',
            'ytick.labelsize' : '40', 'xtick.labelsize':'40' })

sns.lineplot(data=genre_line_plot ,y='quality_streams',x='genre',ax=ax,color="#00e091")

plt.legend([],[], frameon=False)
plt.xlabel('% of Daily Total Streams > 30 Seconds')
plt.ylabel('MAU Monthly Growth Rate')